In [1]:
# By Cristian Perez 
# By Christian Candido

In [21]:
import os
import time
import psutil


# Data Manipulation
import base64
import numpy as np
import pandas as pd 

# LAS 
import laspy
import CSF
from laspy.file import File

# Clustering
import hdbscan

# data visualization
import matplotlib.pyplot as plt
from yellowbrick.cluster import KElbowVisualizer # cluster visualizer
%matplotlib inline

# sklearn kmeans
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import contingency_matrix
from pyclustering.cluster import cluster_visualizer

# Segment-LiDAR
from segment_lidar import samlidar

In [22]:
pc_treesFile = r"C:/Users/pcuser/Documents/Acads/GmE 203/Project/LAS DATA/PC_UPAVE.las"    # Point Cloud w/ Trees
pc_urbanFile = r"C:/Users/pcuser/Documents/Acads/GmE 203/Project/LAS DATA/PC_ZAMBALES.las" # Point Cloud w/ Buildings

In [23]:
las_trees = laspy.read(pc_treesFile) # Read LAS File
las_urban = laspy.read(pc_urbanFile) 
points_trees = las_trees.points      # Access Point Cloud 
points_urban = las_urban.points

In [24]:
point_format = points_trees.point_format # Access properties of point clouds

In [25]:
# Extract X, Y, Z and put into a List 
pc_trees = np.vstack((points_trees.x, points_trees.y, points_trees.z)).transpose() 
pc_urban = np.vstack((points_urban.x, points_urban.y, points_urban.z)).transpose() 

In [26]:
csf = CSF.CSF()

# Parameter Setting
csf.params.bSloopSmooth = False
csf.params.cloth_resolution = 0.50 # more details about parameter: http://ramm.bnu.edu.cn/projects/CSF/download/

##### Point Cloud Dominated with Trees

In [28]:
csf.setPointCloud(pc_trees)

ground = CSF.VecInt() # a list to indicate the index of ground points after calculation
non_ground = CSF.VecInt() # a list to indicate the index of non-ground points after calculation
csf.do_filtering(ground, non_ground)

In [29]:
outFile = laspy.LasData(las_trees.header)
outFile.points = points_trees[np.array(non_ground)] # extract non_ground points, and save it to a las file.

In [30]:
outFile.write(r"C:/Users/pcuser/Documents/Acads/GmE 203/Project/LAS Data/pc_UPAVE_out_non-ground.las")

In [31]:
outPoints = outFile.points

outPoint_cloud = np.vstack((outPoints.x, outPoints.y, outPoints.z, outPoints.red, outPoints.blue, outPoints.green)).transpose() # Extract X, Y, Z and put into a List 
# outPoint_cloud = np.vstack((outPoints.x, outPoints.y, outPoints.z)).transpose()
# outPoint_cloud = np.vstack((outPoints.red, outPoints.blue, outPoints.green)).transpose()

In [32]:
xyz = np.vstack((outPoints.x, outPoints.y, outPoints.z)).transpose()

x=xyz[:,0]
y=xyz[:,1]
z=xyz[:,2]

xmin = np.floor(np.min(x))
ymin = np.floor(np.min(y))
zmin = np.floor(np.min(z))

###### HDBSCan Algorithm

In [69]:
# Start measuring the execution time
start_time = time.time()

# Start measuring the memory usage
start_memory = psutil.Process()

In [70]:
# Set-up parameter for HDBSCAN
clusterer = hdbscan.HDBSCAN(min_cluster_size=100)

# Cluster point clouds using HDBSCAN
labels = clusterer.fit_predict(pc_trees)

In [71]:
# End measuring the execution time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Get the memory usage after running the algorithm
process = psutil.Process(os.getpid())
memory_info = process.memory_info()
memory_usage = memory_info.rss / (1024 * 1024)  # Convert to megabytes

# Print the results
print("Elapsed Time: {:.6f} seconds".format(elapsed_time))
print("Memory Usage: {:.2f} MB".format(memory_usage))

Elapsed Time: 51.983993 seconds
Memory Usage: 3430.33 MB


In [72]:
print ("No. of Clusters: ", len(np.unique(labels)))

No. of Clusters:  45


In [73]:
# Create a new header for clustered label
header = laspy.LasHeader(point_format=7, version="1.4") 
header.add_extra_dim(laspy.ExtraBytesParams(name="Clusters", type=np.int32))

In [76]:
las = laspy.LasData(header)
las.header.offset = [xmin,ymin,zmin]
las.header.scale = [0.01,0.01,0.01]
las.x = points_trees.x     # x-coordinate
las.y = points_trees.y     # y-coordinate
las.z = points_trees.z     # z-coordinate
las.Clusters = labels   # clustered label
las.write(r"C:/Users/pcuser/Documents/Acads/GmE 203/Project/LAS Data/classified_hdbscan_pc_upave.las") # write las file 

###### Segment-LiDAR

In [42]:
# Start measuring the execution time
start_time = time.time()

# Start measuring the memory usage
start_memory = psutil.Process()

In [43]:
out = r"C:/Users/pcuser/Documents/Acads/GmE 203/Project/LAS Data/classified_segment-lidar_pc_upave.las"

model = samlidar.SamLidar(ckpt_path="sam_vit_h_4b8939.pth")
points = model.read(pc_treesFile)
cloud, non_ground, ground = model.csf(points)
labels, *_ = model.segment(points=cloud, image_path="raster.tif", labels_path="labeled.tif")
model.write(points=points, non_ground=non_ground, ground=ground, segment_ids=labels, save_path=out)

Reading C:/Users/pcuser/Documents/Acads/GmE 203/Project/LAS DATA/PC_UPAVE.las...
- Classification value is not provided. Reading all points...
- Reading RGB values...
File reading is completed in 0.04 seconds. The point cloud contains 858253 points.

Applying CSF algorithm...
CSF algorithm is completed in 36.92 seconds. The filtered non-ground cloud contains 432399 points.

Segmenting the point cloud...
- Generating raster image...
- Saving raster image...
- Applying segment-geospatial to raster image...
- Saving segmented image...
- Generating segment IDs...
Segmentation is completed in 73.45 seconds. Number of instances: 110

Writing the segmented point cloud to C:/Users/pcuser/Documents/Acads/GmE 203/Project/LAS Data/classified_segment-lidar_pc_upave.las...
Writing is completed in 0.24 seconds.



In [45]:
print ("No. of Clusters: ", len(np.unique(labels)))

No. of Clusters:  44


In [46]:
# End measuring the execution time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Get the memory usage after running the algorithm
process = psutil.Process(os.getpid())
memory_info = process.memory_info()
memory_usage = memory_info.rss / (1024 * 1024)  # Convert to megabytes

# Print the results
print("Elapsed Time: {:.6f} seconds".format(elapsed_time))
print("Memory Usage: {:.2f} MB".format(memory_usage))

Elapsed Time: 967.809243 seconds
Memory Usage: 3310.18 MB


##### Point Cloud Dominated with Buildings

In [51]:
csf.setPointCloud(pc_urban)

ground = CSF.VecInt() # a list to indicate the index of ground points after calculation
non_ground = CSF.VecInt() # a list to indicate the index of non-ground points after calculation
csf.do_filtering(ground, non_ground)

In [52]:
outFile = laspy.LasData(las_urban.header)
outFile.points = points_urban[np.array(non_ground)] # extract non_ground points, and save it to a las file.

In [53]:
outFile.write(r"C:/Users/pcuser/Documents/Acads/GmE 203/Project/LAS Data/pc_ZAM_out_non-ground.las")

In [54]:
outPoints = outFile.points

outPoint_cloud = np.vstack((outPoints.x, outPoints.y, outPoints.z, outPoints.red, outPoints.blue, outPoints.green)).transpose() # Extract X, Y, Z and put into a List 
# outPoint_cloud = np.vstack((outPoints.x, outPoints.y, outPoints.z)).transpose()
# outPoint_cloud = np.vstack((outPoints.red, outPoints.blue, outPoints.green)).transpose()

In [55]:
xyz = np.vstack((outPoints.x, outPoints.y, outPoints.z)).transpose()

x=xyz[:,0]
y=xyz[:,1]
z=xyz[:,2]

xmin = np.floor(np.min(x))
ymin = np.floor(np.min(y))
zmin = np.floor(np.min(z))

###### HDBSCan Algorithm

In [77]:
# Start measuring the execution time
start_time = time.time()

# Start measuring the memory usage
start_memory = psutil.Process()

In [78]:
# Set-up parameter for HDBSCAN
clusterer = hdbscan.HDBSCAN(min_cluster_size=100)

# Cluster point clouds using HDBSCAN
labels = clusterer.fit_predict(pc_urban)

In [79]:
# End measuring the execution time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Get the memory usage after running the algorithm
process = psutil.Process(os.getpid())
memory_info = process.memory_info()
memory_usage = memory_info.rss / (1024 * 1024)  # Convert to megabytes

# Print the results
print("Elapsed Time: {:.6f} seconds".format(elapsed_time))
print("Memory Usage: {:.2f} MB".format(memory_usage))

Elapsed Time: 249.620659 seconds
Memory Usage: 3575.50 MB


In [80]:
print ("No. of Clusters: ", len(np.unique(labels)))

No. of Clusters:  241


In [81]:
# Create a new header for clustered label
header = laspy.LasHeader(point_format=7, version="1.4") 
header.add_extra_dim(laspy.ExtraBytesParams(name="Clusters", type=np.int32))

In [82]:
las = laspy.LasData(header)
las.header.offset = [xmin,ymin,zmin]
las.header.scale = [0.01,0.01,0.01]
las.x = points_urban.x     # x-coordinate
las.y = points_urban.y     # y-coordinate
las.z = points_urban.z     # z-coordinate
las.Clusters = labels   # clustered label
las.write(r"C:/Users/pcuser/Documents/Acads/GmE 203/Project/LAS Data/classified_hdbscan_pc_zam.las") # write las file 

###### Segment-LiDAR

In [62]:
# Start measuring the execution time
start_time = time.time()

# Start measuring the memory usage
start_memory = psutil.Process()

In [63]:
out = r"C:/Users/pcuser/Documents/Acads/GmE 203/Project/LAS Data/classified_segment-lidar_pc_zam.las"

model = samlidar.SamLidar(ckpt_path="sam_vit_h_4b8939.pth")
points = model.read(pc_urbanFile)
cloud, non_ground, ground = model.csf(points)
labels, *_ = model.segment(points=cloud, image_path="raster.tif", labels_path="labeled.tif")
model.write(points=points, non_ground=non_ground, ground=ground, segment_ids=labels, save_path=out)

Reading C:/Users/pcuser/Documents/Acads/GmE 203/Project/LAS DATA/PC_ZAMBALES.las...
- Classification value is not provided. Reading all points...
- Reading RGB values...
File reading is completed in 0.09 seconds. The point cloud contains 2200062 points.

Applying CSF algorithm...
CSF algorithm is completed in 12.45 seconds. The filtered non-ground cloud contains 857309 points.

Segmenting the point cloud...
- Generating raster image...
- Saving raster image...
- Applying segment-geospatial to raster image...
- Saving segmented image...
- Generating segment IDs...
Segmentation is completed in 69.65 seconds. Number of instances: 115

Writing the segmented point cloud to C:/Users/pcuser/Documents/Acads/GmE 203/Project/LAS Data/classified_segment-lidar_pc_zam.las...
Writing is completed in 0.63 seconds.



In [64]:
# End measuring the execution time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Get the memory usage after running the algorithm
process = psutil.Process(os.getpid())
memory_info = process.memory_info()
memory_usage = memory_info.rss / (1024 * 1024)  # Convert to megabytes

# Print the results
print("Elapsed Time: {:.6f} seconds".format(elapsed_time))
print("Memory Usage: {:.2f} MB".format(memory_usage))

Elapsed Time: 87.474845 seconds
Memory Usage: 3477.61 MB


In [65]:
print ("No. of Clusters: ", len(np.unique(labels)))

No. of Clusters:  90


In [84]:
data = {'Model':['HDBSCAN_UP-AVE_NON-G', 'HDBSCAN_ZAMBALES_NON-G', 'HDBSCAN_UP-AVE', 'HDBSCAN_ZAMBALES','SEGMENT-LIDAR_UP-AVE', 'SEGMENT-LIDAR_ZAMBALES'],\
        'NO. OF POINT CLOUDS: ':[449527, 885221, 858253, 2200062, 858253, 2200062],\
        'NO. OF CLUSTERS':[8, 1140, 45, 241, 44, 90],\
        'ELAPSED TIME (S)':[19.26466, 46.42636, 51.983993, 249.620659, 115.0000, 86.2000],\
        'MEMORY USAGE (MB): ': [3307.43, 3425.13, 3430.33, 3575.50, 3310.18, 3477.61]}

df = pd.DataFrame(data)

df

,Model,NO. OF POINT CLOUDS:,NO. OF CLUSTERS,ELAPSED TIME (S),MEMORY USAGE (MB):
0,HDBSCAN_UP-AVE_NON-G,449527,8,19.264660,3307.43
1,HDBSCAN_ZAMBALES_NON-G,885221,1140,46.426360,3425.13
2,HDBSCAN_UP-AVE,858253,45,51.983993,3430.33
3,HDBSCAN_ZAMBALES,2200062,241,249.620659,3575.50
4,SEGMENT-LIDAR_UP-AVE,858253,44,115.000000,3310.18
5,SEGMENT-LIDAR_ZAMBALES,2200062,90,86.200000,3477.61
